In [1]:
from pathlib import Path
import os, random, numpy as np
from PIL import Image
from tqdm import tqdm

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

DEVICE = "cpu"
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

NB_DIR = Path.cwd()
PROJ_ROOT = NB_DIR.parent
DATA_DIR = PROJ_ROOT / "data"
TRAIN_DIR = DATA_DIR / "classification_data" / "train_data"
VAL_DIR   = DATA_DIR / "classification_data" / "val_data"
MODEL_DIR = PROJ_ROOT / "models"

print("Running on:", DEVICE)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\study\cos30082\final_asm\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\study\cos30082\final_asm\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\study\cos30082\final_asm\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "d:\study\co

Running on: cpu


In [ ]:
IMG_SIZE = 160
BATCH_SIZE = 16

tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


In [4]:
class TripletFaceDataset(Dataset):
    def __init__(self, root, transform):
        self.transform = transform
        self.id2imgs = {}
        for person in sorted(os.listdir(root)):
            pdir = os.path.join(root, person)
            if not os.path.isdir(pdir): continue
            imgs = list(Path(pdir).glob("*"))
            if len(imgs) >= 2:
                self.id2imgs[person] = imgs
        self.ids = list(self.id2imgs.keys())

    def __len__(self):  # approximate
        return len(self.ids) * 4

    def __getitem__(self, idx):
        anchor_id = random.choice(self.ids)
        pos_id = anchor_id
        neg_id = random.choice([i for i in self.ids if i != anchor_id])

        a, p = random.sample(self.id2imgs[anchor_id], 2)
        n = random.choice(self.id2imgs[neg_id])

        def load(path): return self.transform(Image.open(path).convert("RGB"))
        return load(a), load(p), load(n)

triplet_ds = TripletFaceDataset(TRAIN_DIR, tf)
triplet_loader = DataLoader(triplet_ds, batch_size=BATCH_SIZE, shuffle=True)


In [5]:
from torchvision.models import ResNet18_Weights

class TripletEmbed(nn.Module):
    def __init__(self, emb_dim=128):
        super().__init__()
        backbone = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        feat_dim = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.backbone = backbone
        self.embed = nn.Linear(feat_dim, emb_dim)

    def forward(self, x):
        f = self.backbone(x)
        e = nn.functional.normalize(self.embed(f))
        return e

EMB_DIM = 128
model = TripletEmbed(emb_dim=EMB_DIM).to(DEVICE)

# Load your fine-tuned checkpoint weights for backbone
FINETUNED_PATH = MODEL_DIR / "classifier_embed_resnet18_softmax_cpu.pt"
state = torch.load(FINETUNED_PATH, map_location=DEVICE)
backbone_dict = {k.replace("backbone.", ""): v for k, v in state.items() if "backbone." in k}
model.backbone.load_state_dict(backbone_dict, strict=False)
print("Loaded fine-tuned backbone weights")


Loaded fine-tuned backbone weights


In [6]:
def triplet_loss(a, p, n, margin=0.2):
    d_ap = (a - p).pow(2).sum(1)
    d_an = (a - n).pow(2).sum(1)
    return torch.relu(d_ap - d_an + margin).mean()

optimizer = optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    for a, p, n in tqdm(triplet_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", ncols=90):
        a, p, n = a.to(DEVICE), p.to(DEVICE), n.to(DEVICE)
        optimizer.zero_grad()
        ea, ep, en = model(a), model(p), model(n)
        loss = triplet_loss(ea, ep, en)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg = epoch_loss / len(triplet_loader)
    print(f"Epoch {epoch+1}: triplet_loss={avg:.4f}")

torch.save(model.state_dict(), MODEL_DIR / "triplet_embed_resnet18_cpu.pt")
print("Saved triplet embedding model")

Epoch 1/5: 100%|██████████████████████████████████████| 1000/1000 [26:40<00:00,  1.60s/it]


Epoch 1: triplet_loss=0.0153


Epoch 2/5: 100%|██████████████████████████████████████| 1000/1000 [25:18<00:00,  1.52s/it]


Epoch 2: triplet_loss=0.0159


Epoch 3/5: 100%|██████████████████████████████████████| 1000/1000 [25:03<00:00,  1.50s/it]


Epoch 3: triplet_loss=0.0168


Epoch 4/5: 100%|██████████████████████████████████████| 1000/1000 [24:29<00:00,  1.47s/it]


Epoch 4: triplet_loss=0.0155


Epoch 5/5: 100%|██████████████████████████████████████| 1000/1000 [23:53<00:00,  1.43s/it]

Epoch 5: triplet_loss=0.0161
Saved triplet embedding model
